---

# Notebook : Optimisation des hyperparamètres avec Grid Search pour Random Forest

**Objectifs :**
1. Utiliser **Grid Search** pour optimiser les hyperparamètres du modèle **Random Forest** en explorant différentes valeurs de `n_estimators`, `max_depth`, `min_samples_split` et `min_samples_leaf`.
2. Entraîner le modèle optimisé sur un échantillon de **20% des données**.
3. Évaluer les performances du modèle optimisé et sauvegarder les résultats, avec une vérification de seuil de performance.

---

## Étape 1 : Importer les bibliothèques nécessaires et charger les données


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
import json

---

## Étape 2 : Charger les données et préparer un échantillon de 20%

In [4]:
# Charger les données complètes
file_path = 'note-books/phase_2_training/04_correlation_analysis/data_final_features.csv'
df = pd.read_csv(file_path)

# Encoder `nutriscore_grade` en valeurs numériques
df['nutriscore_grade_encoded'] = df['nutriscore_grade'].map({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})

# Préparer les variables explicatives et la cible en utilisant les colonnes disponibles et validées
X = df[['fat_100g', 'saturated-fat_100g', 'energy-kcal_100g', 'sugars_100g', 'salt_100g', 'proteins_100g']]
y = df['nutriscore_grade_encoded']

# Prendre un échantillon de 20% des données pour l’entraînement
X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.8, random_state=42)


---

## Étape 3 : Définir la pipeline et les hyperparamètres pour Grid Search

Création d'une pipeline avec une étape de standardisation et le modèle **Random Forest**, suivie de la définition des hyperparamètres pour le **Grid Search**.

In [5]:
# Charger le modèle Random Forest de base
random_forest_model = RandomForestRegressor(random_state=42)

# Créer une pipeline avec une étape de standardisation et le modèle Random Forest
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', random_forest_model)
])

# Définir la grille de recherche des hyperparamètres
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

---

## Étape 4 : Optimiser les hyperparamètres avec GridSearchCV

Utilisation de **GridSearchCV** pour explorer toutes les combinaisons d’hyperparamètres.

In [6]:
# Configurer GridSearchCV pour la recherche d'hyperparamètres
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Entraîner le modèle avec Grid Search sur l'échantillon de 20% des données
grid_search.fit(X_sample, y_sample)

# Extraire les meilleurs paramètres
best_params = grid_search.best_params_
print("Meilleurs paramètres trouvés avec Grid Search :", best_params)

# Sauvegarder les meilleurs paramètres pour un suivi ultérieur
output_folder = 'note-books/phase_2_training/08_random_forest_optimized'
os.makedirs(output_folder, exist_ok=True)
hyperparams_path = f'{output_folder}/best_hyperparameters.json'
with open(hyperparams_path, 'w') as f:
    json.dump(best_params, f)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   2.3s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   2.2s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   2.4s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   2.3s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   2.4s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=   4.5s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=   4.6s
[CV] END model__max_depth=10, model__min

---

## Étape 5 : Évaluer les performances du modèle optimisé

Évaluation des performances du modèle optimisé et vérification d'un seuil de performance (par exemple, R² ≥ 0.95).

In [7]:
# Utiliser le meilleur modèle trouvé par Grid Search
best_model = grid_search.best_estimator_

# Prédire sur les données d'entraînement pour évaluer les performances
y_pred = best_model.predict(X_sample)

# Calculer les métriques de performance
mse = mean_squared_error(y_sample, y_pred)
r2 = r2_score(y_sample, y_pred)

print(f"Évaluation des performances du modèle optimisé avec 20% des données :")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")

# Vérification du seuil de performance
if r2 >= 0.95:
    print("Le modèle atteint le seuil de performance souhaité (R² ≥ 0.95).")
else:
    print("Le modèle n'atteint pas le seuil de performance souhaité. Envisagez une optimisation supplémentaire.")

Évaluation des performances du modèle optimisé avec 20% des données :
Mean Squared Error (MSE) : 0.13488161591349912
Coefficient de détermination (R²) : 0.9253878795937978
Le modèle n'atteint pas le seuil de performance souhaité. Envisagez une optimisation supplémentaire.


---

## Étape 6 : Sauvegarder le modèle optimisé et l'objet GridSearchCV

Si les performances sont satisfaisantes, nous sauvegardons le modèle optimisé ainsi que l’objet `GridSearchCV` pour un suivi complet.

In [8]:
# Chemins de sauvegarde
model_path_optimized = f'{output_folder}/Random_Forest_20_percent_optimized.joblib'
grid_search_path = f'{output_folder}/grid_search_object.joblib'

# Sauvegarder le modèle optimisé
joblib.dump(best_model, model_path_optimized)
print(f"Modèle Random Forest optimisé sauvegardé sous : {model_path_optimized}")

# Sauvegarder l'objet GridSearchCV pour les détails d'optimisation
joblib.dump(grid_search, grid_search_path)
print(f"Objet GridSearchCV sauvegardé sous : {grid_search_path}")

Modèle Random Forest optimisé sauvegardé sous : note-books/phase_2_training/08_random_forest_optimized/Random_Forest_20_percent_optimized.joblib
Objet GridSearchCV sauvegardé sous : note-books/phase_2_training/08_random_forest_optimized/grid_search_object.joblib



---

### Explication des étapes

1. **Optimisation avec Grid Search** : Utilisation de `GridSearchCV` pour explorer systématiquement les valeurs d’hyperparamètres et identifier les meilleurs paramètres.
2. **Évaluation des performances avec vérification de seuil** : Calcul du MSE et du R² pour vérifier si le modèle atteint le seuil de performance souhaité (par exemple, R² ≥ 0.95).
3. **Sauvegarde avancée** :
   - **Meilleurs paramètres** : Sauvegardés dans un fichier JSON pour un suivi précis.
   - **Modèle optimisé** : Sauvegardé pour une utilisation future.
   - **Objet GridSearchCV** : Sauvegardé pour accéder aux détails complets de l’optimisation et des combinaisons d’hyperparamètres testées.

---

### Résumé

Dans ce notebook, nous avons :
1. Utilisé **Grid Search** pour optimiser les hyperparamètres du modèle **Random Forest** sur un échantillon de 20% des données.
2. Évalué les performances du modèle optimisé, en vérifiant un seuil de performance pour R².
3. Sauvegardé le modèle optimisé, les meilleurs hyperparamètres, et l'objet `GridSearchCV` pour une traçabilité complète et des ajustements futurs.

